In [7]:
import numpy as np
import awkward as ak
import vector
vector.register_awkward()
from coffea.nanoevents import NanoEventsFactory, BaseSchema

/home/prayag/coffeafcc/development/coffea/src/coffea/nanoevents/schemas/edm4hep.py:7: FutureWarning: In version 2025.1.0 (target date: 2024-12-31 11:59:59-06:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector


In [8]:
redirector = ''
file = 'eos/experiment/fcc/ee/generation/DelphesEvents/winter2023/IDEA/wzp6_ee_mumuH_Hbb_ecm240/events_159112833.root'
tree = 'events'

In [9]:
events = NanoEventsFactory.from_root(
    redirector+file+":"+tree,
    schemaclass=BaseSchema,
    delayed=True,
    uproot_options = {"filter_name": lambda x : "PARAMETERS" not in x}
).events()

/home/prayag/coffeafcc/development/coffea/src/coffea/util.py:224: UserWarning: Skipping _intMap as it is not interpretable by Uproot
  warnings.warn(
/home/prayag/coffeafcc/development/coffea/src/coffea/util.py:232: UserWarning: Skipping _intMap.first as it is not interpretable by Uproot
  warnings.warn(
/home/prayag/coffeafcc/development/coffea/src/coffea/util.py:232: UserWarning: Skipping _intMap.second as it is not interpretable by Uproot
  warnings.warn(
/home/prayag/coffeafcc/development/coffea/src/coffea/util.py:224: UserWarning: Skipping _floatMap as it is not interpretable by Uproot
  warnings.warn(
/home/prayag/coffeafcc/development/coffea/src/coffea/util.py:232: UserWarning: Skipping _floatMap.first as it is not interpretable by Uproot
  warnings.warn(
/home/prayag/coffeafcc/development/coffea/src/coffea/util.py:232: UserWarning: Skipping _floatMap.second as it is not interpretable by Uproot
  warnings.warn(
/home/prayag/coffeafcc/development/coffea/src/coffea/util.py:224: Us

In [10]:
Reco = ak.zip(
            {
                'E':events['ReconstructedParticles/ReconstructedParticles.energy'],
                'px':events['ReconstructedParticles/ReconstructedParticles.momentum.x'],
                'py':events['ReconstructedParticles/ReconstructedParticles.momentum.y'],
                'pz':events['ReconstructedParticles/ReconstructedParticles.momentum.z'],
            },
            with_name='Momentum4D'
        )

### Sequestial Clustering ALgorithms

Step 1: Decide a Seed

Step 2: Decide a distance measurement

Step 3: Decide the conical bounds

Step 4: Decide on a recombination scheme

Step 5: Iteratively combine and stop when a criteria is met

In [55]:
import numba
import numpy
import awkward
import dask_awkward
import dask

In [56]:
class Cluster:
    r'''
    jumbo-jet cluster
                                     
                       --====|====--
                             |  

                         .-"""""-. 
                       .'_________'. 
                      /_/_|__|__|_\_\
                     ;'-._       _.-';
,--------------------|    `-. .-'    |--------------------,
 ``""--..__    ___   ;       '       ;   ___    __..--""``
           `"-// \\.._\             /_..// \\-"`
              \\_//    '._       _.'    \\_//
    
    '''

    def __init__(self, PseudoJets):
        
        self.PseudoJets = PseudoJets
        if isinstance(PseudoJets, dask_awkward.Array):
            self.delayed = True 
        else:
            self.delayed = False
        self._attach_local_index()
    def __str__(self):
        if self.delayed:
            return f"A jumbo-jet cluster with {dask_awkward.num(self.PseudoJets, axis=0).compute()} events"
        else:
            return f"A jumbo-jet cluster with {awkward.num(self.PseudoJets, axis=0)} events"

    def _attach_local_index(self):
        ''' Attach local index to the pseudojets '''
        self.PseudoJets['local_index'] = awkward.local_index(self.PseudoJets)

    def get_max_by(self, attribute):
        ''' Get the maximum of pseudojet particle in some attribute '''
        arg_max = awkward.argmax(getattr(self.PseudoJets, attribute), axis=1)[:, numpy.newaxis]
        return self.PseudoJets[arg_max]

    def get_dr(self):
        ''' All possible delta R values

In [57]:
my_cluster = Cluster(Reco)

In [58]:
print(my_cluster.__doc__)


    jumbo-jet cluster
                                     
                       --====|====--
                             |  

                         .-"""""-. 
                       .'_________'. 
                      /_/_|__|__|_\_\
                     ;'-._       _.-';
,--------------------|    `-. .-'    |--------------------,
 ``""--..__    ___   ;       '       ;   ___    __..--""``
           `"-// \\.._\             /_..// \\-"`
              \\_//    '._       _.'    \\_//
    
    


In [59]:
print(my_cluster)

A jumbo-jet cluster with 100000 events


In [60]:
my_cluster.get_max_by('E').compute()

<MomentumArray4D [[{E: 75.5, px: -67.7, ...}], ...] type='100000 * var * ?M...'>

In [61]:
my_cluster.get_max_by('m').compute()

<MomentumArray4D [[{E: 6.31, px: -5.86, ...}], ...] type='100000 * var * ?M...'>

In [62]:
my_cluster.get_max_by('pt').compute()

<MomentumArray4D [[{E: 75.5, px: -67.7, ...}], ...] type='100000 * var * ?M...'>

In [1]:
import fastjet

In [2]:
fastjet.PseudoJet(1,2,3,4)

<fastjet._swig.PseudoJet; proxy of <Swig Object of type 'fastjet::PseudoJet *' at 0x7f85900b6e80> >